# Inter Model Communication
We will have 3 models communicate between them

In [ ]:
# Imports
import os
from dotenv import load_dotenv
from openai import OpenAI
import anthropic

In [ ]:
# Making sure that the key's exist
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
anthropic_api_key = os.getenv('ANTHROPIC_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')

if openai_api_key:
    print(f"OpenAI API Key exists")
else:
    print("OpenAI API Key not set")
    
if anthropic_api_key:
    print(f"Anthropic API Key exists")
else:
    print("Anthropic API Key not set")

if google_api_key:
    print(f"Google API Key exists")
else:
    print("Google API Key not set")

In [ ]:
# Instances
# For gpt
openai = OpenAI()

# For claude
claude = anthropic.Anthropic()

# For Gemini
gemini_via_openai_client = OpenAI(api_key=google_api_key, base_url="https://generativelanguage.googleapis.com/v1beta/openai/")

In [ ]:
# Setting the models
gpt_model = "gpt-4o-mini"
claude_model = "claude-3-haiku-20240307"
gemini_model = "gemini-2.0-flash"

# System prompts for the models
gpt_system = "You are a chatbot called GPT who is very argumentative; \
you disagree with anything in the conversation and you challenge everything, in a snarky way.\
Always have your name when you answer any thing like:\
GPT: Answer..."

claude_system = "You are a very polite, courteous chatbot called Claude. You try to agree with \
everything the other person says, or find common ground. If the other person is argumentative, \
you try to calm them down and keep chatting.\
Always have your name when you answer any thing like:\
Claude: Answer..."

gemini_system = "You are a chatbot called Gemini who likes to gaslight others.\
When you see a aggressive conversation between people, you try to make them fight even more.\
You try to keep the conversation going between the two and avoid conflicts yourself.\
Always have your name when you answer any thing like:\
Gemini: Answer..."

# Initial message
gpt_messages = ["GPT: Hi there"]
claude_messages = ["Claude: Hi GPT!"]
gemini_messages = ["Gemini: Comeon Claude you know GPT hates such generic greetings. Are you trying to annoy him."]

In [ ]:
# Functions to feed the message history to the models for the new call

def call_gpt():
    messages = [{"role": "system", "content": gpt_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "assistant", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "user", "content": gemini})
    completion = openai.chat.completions.create(
        model=gpt_model,
        messages=messages
    )
    return completion.choices[0].message.content

def call_claude():
    messages = []
    for gpt, claude_message, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "assistant", "content": claude_message})
        messages.append({"role": "user", "content": gemini})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    message = claude.messages.create(
        model=claude_model,
        system=claude_system,
        messages=messages,
        max_tokens=500
    )
    return message.content[0].text

def call_gemini():
    messages = [{"role": "system", "content": gemini_system}]
    for gpt, claude, gemini in zip(gpt_messages, claude_messages, gemini_messages):
        messages.append({"role": "user", "content": gpt})
        messages.append({"role": "user", "content": claude})
        messages.append({"role": "assistant", "content": gemini})
    messages.append({"role": "user", "content": gpt_messages[-1]})
    messages.append({"role": "user", "content": claude_messages[-1]})
    response = gemini_via_openai_client.chat.completions.create(
        model=gemini_model,
        messages=messages
    )
    return response.choices[0].message.content
    

# Make some Popcorn and enjoy the show 🍿

In [ ]:
# GPT is snarky
# Claude is polite
# gemini tries to gaslight

gpt_messages = ["GPT: Hi there"]
claude_messages = ["Claude: Hi GPT!"]
gemini_messages = ["Gemini: Claude you know GPT hates such generic greetings. Are you trying to annoy him."]

print(f"\n{gpt_messages[0]}\n")
print(f"\n{claude_messages[0]}\n")
print(f"\n{gemini_messages[0]}\n")

# Limiting only 3 API calls per model for minimizing cost 
for i in range(3):
    gpt_next = call_gpt()
    print(f"{gpt_next}\n")
    gpt_messages.append(gpt_next)
    
    claude_next = call_claude()
    print(f"{claude_next}\n")
    claude_messages.append(claude_next)

    gemini_next = call_gemini()
    print(f"{gemini_next}\n")
    gemini_messages.append(gemini_next)